# ChronoNet

Here we train the model and record the train and validation accuracy. We used subset of the temple dataset for faster training. we used 500 subjects only for training and validation of the model. The recorded metrics are accuracy, binary cross entropy loss, F1 score, Precision and Recall.

- Train Data: 500 subjects

- Test Data: 276 subjects

In [26]:
!ls tuh/train/normal/* | wc -l

    1371


In [1]:
import torch
from tqdm import tqdm
import torch.nn as nn

from load_data import read_data_arrays, data_file_names, standardize_data, data_loader
from models import ChronoNet
from utils import cal_accuracy, evaluate_model 


BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 5

print("Reading Data....")
sample_size = 500
data_files = data_file_names(sample_size)
(train_features, val_features, test_features,
 train_labels, val_labels, test_labels, test_lengths) = read_data_arrays(
    data_files)
    
print("Scaling Data....")
train_features, val_features, test_features = standardize_data(
    train_features, val_features, test_features)
    
print("Data Loader....")
train_iter = data_loader(train_features, train_labels, DEVICE, BATCH_SIZE)
val_iter = data_loader(val_features, val_labels, DEVICE, BATCH_SIZE)
test_iter = data_loader(test_features, test_labels, DEVICE, BATCH_SIZE, shuffle=False)
    
print("Training Model....")
n_chans = 19
model=ChronoNet(n_chans)
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, val_iter)
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        cal_accuracy(model, train_iter)[0], ",F1: ", 
        cal_accuracy(model, train_iter)[4], ",Precision: ", 
        cal_accuracy(model, train_iter)[2], ",Recall: ", 
        cal_accuracy(model, train_iter)[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        cal_accuracy(model, val_iter)[0], ",F1: ", 
        cal_accuracy(model, val_iter)[4], ",Precision: ", 
        cal_accuracy(model, val_iter)[2], ",Recall: ", 
        cal_accuracy(model, val_iter)[3])

/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading Data....
Scaling Data....
Data Loader....
Training Model....
Epoch 1


100%|█████████████████████████████████████████| 692/692 [04:34<00:00,  2.52it/s]


Train loss: 0.6135080639337529 ,Train Accuracy:  0.7996045197740113 ,F1:  0.7803633571525878 ,Precision:  0.8632490341671918 ,Recall:  0.712
Val loss: 0.6022120368429077 , Val Accuracy:  0.7723050847457628 ,F1:  0.741564387672656 ,Precision:  0.8573080686771639 ,Recall:  0.6533559322033898
Epoch 2


100%|█████████████████████████████████████████| 692/692 [04:21<00:00,  2.65it/s]


...

Train loss: 0.5753913293511881 ,Train Accuracy:  0.8453446327683616 ,F1:  0.8308555469049296 ,Precision:  0.9167416618942431 ,Recall:  0.7596836158192091
Val loss: 0.5960082494335257 , Val Accuracy:  0.792 ,F1:  0.7715221924337206 ,Precision:  0.8557739963654386 ,Recall:  0.7023728813559322
Epoch 4


100%|█████████████████████████████████████████| 692/692 [04:30<00:00,  2.55it/s]


Train loss: 0.570163000353499 ,Train Accuracy:  0.8605762711864406 ,F1:  0.8510484192228297 ,Precision:  0.9134727513021845 ,Recall:  0.7966101694915254
Val loss: 0.595359214972624 , Val Accuracy:  0.7965084745762712 ,F1:  0.7804717498628635 ,Precision:  0.8472409686383485 ,Recall:  0.723457627118644
Epoch 5


100%|█████████████████████████████████████████| 692/692 [04:15<00:00,  2.71it/s]


Train loss: 0.565777277498576 ,Train Accuracy:  0.8552542372881355 ,F1:  0.8381267691063485 ,Precision:  0.9506105601100728 ,Recall:  0.7494463276836159
Val loss: 0.5963385198023412 , Val Accuracy:  0.7821016949152543 ,F1:  0.7513346228239846 ,Precision:  0.8748648648648648 ,Recall:  0.6583728813559322


## Testing the model

In [2]:
from collections import Counter
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)
        
y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
print("Test Accuracy: ", accuracy_score(y_final, yhat_final))
print("Test Confusion: ",confusion_matrix(y_final, yhat_final))
print("Test F1: ", f1_score(y_final, yhat_final))
print("Test Recall: ", recall_score(y_final, yhat_final))
print("Test Precision: ", precision_score(y_final, yhat_final))

Test Accuracy:  0.8188405797101449
Test Confusion:  [[141   9]
 [ 41  85]]
Test F1:  0.7727272727272727
Test Recall:  0.6746031746031746
Test Precision:  0.9042553191489362


# LSTM

In [5]:
from models import LSTM

learning_rate = 5e-3

model = LSTM(input_size=500, num_channels=19, hidden_units=128)

print("Training Model....")
n_chans = 19
DEVICE = torch.device("cpu")
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
NUM_EPOCHS = 5
BATCH_SIZE = 128

print("started training")

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, test_iter)
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        cal_accuracy(model, train_iter)[0], ",F1: ", 
        cal_accuracy(model, train_iter)[4], ",Precision: ", 
        cal_accuracy(model, train_iter)[2], ",Recall: ", 
        cal_accuracy(model, train_iter)[3])
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        cal_accuracy(model, val_iter)[0], ",F1: ", 
        cal_accuracy(model, val_iter)[4], ",Precision: ", 
        cal_accuracy(model, val_iter)[2], ",Recall: ", 
        cal_accuracy(model, val_iter)[3])

Training Model....
started training
Epoch 1


100%|█████████████████████████████████████████| 692/692 [08:51<00:00,  1.30it/s]


Train loss: 0.661770528279288 ,Train Accuracy:  0.7018870056497175 ,F1:  0.624489389259739 ,Precision:  0.843477257872275 ,Recall:  0.49577401129943505
Val loss: 0.67322963032068 , Val Accuracy:  0.5675593220338983 ,F1:  0.2789803877239586 ,Precision:  0.8386000679578661 ,Recall:  0.16732203389830508
Epoch 2


100%|█████████████████████████████████████████| 692/692 [36:10<00:00,  3.14s/it]


Train loss: 0.6248360861071273 ,Train Accuracy:  0.7378757062146892 ,F1:  0.6679738936279842 ,Precision:  0.9108829729096729 ,Recall:  0.5273446327683616
Val loss: 0.6709002801016265 , Val Accuracy:  0.5743050847457627 ,F1:  0.2901074053137365 ,Precision:  0.8727891156462585 ,Recall:  0.17396610169491525
Epoch 3


...

Train loss: 0.6036012842820558 ,Train Accuracy:  0.7931073446327683 ,F1:  0.7555798803929944 ,Precision:  0.9229991520448764 ,Recall:  0.6395706214689265
Val loss: 0.6608413919514301 , Val Accuracy:  0.602 ,F1:  0.3752461022721226 ,Precision:  0.8721246599060104 ,Recall:  0.2390508474576271
Epoch 4


100%|█████████████████████████████████████████| 692/692 [13:37<00:00,  1.18s/it]


Train loss: 0.5900325086075446 ,Train Accuracy:  0.8101694915254237 ,F1:  0.777824798984342 ,Precision:  0.937575718931327 ,Recall:  0.6645875706214689
Val loss: 0.6607672583823111 , Val Accuracy:  0.604 ,F1:  0.37907940895078135 ,Precision:  0.8774606299212598 ,Recall:  0.2417627118644068
Epoch 5


100%|█████████████████████████████████████████| 692/692 [17:01<00:00,  1.48s/it]


Train loss: 0.5812325871231928 ,Train Accuracy:  0.833683615819209 ,F1:  0.8115485564304462 ,Precision:  0.9361394181066313 ,Recall:  0.716225988700565
Val loss: 0.6539293941329507 , Val Accuracy:  0.6249830508474576 ,F1:  0.44049967126890205 ,Precision:  0.8670117459685447 ,Recall:  0.2952542372881356


In [6]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)

y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
print("Test Accuracy: ", accuracy_score(y_final, yhat_final))
print("Test Confusion: ",confusion_matrix(y_final, yhat_final))
print("Test F1: ", f1_score(y_final, yhat_final))
print("Test Recall: ", recall_score(y_final, yhat_final))
print("Test Precision: ", precision_score(y_final, yhat_final))

Test Accuracy:  0.6485507246376812
Test Confusion:  [[149   1]
 [ 96  30]]
Test F1:  0.38216560509554137
Test Recall:  0.23809523809523808
Test Precision:  0.967741935483871


# GCN

In [269]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from tqdm import tqdm
import torch.nn as nn
import torch
from torch.autograd import Variable


class Block(nn.Module):
  def __init__(self,inplace):
    super().__init__()
    self.conv1=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=2,stride=2,padding=0)
    self.conv2=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=4,stride=2,padding=1)
    self.conv3=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=8,stride=2,padding=3)
    self.relu=nn.ReLU()

  def forward(self,x):
    x1=self.relu(self.conv1(x))
    x2=self.relu(self.conv2(x))
    x3=self.relu(self.conv3(x))
    x=torch.cat([x1,x3,x3],dim=1)
    return x

class ChronoNet(nn.Module):
  def __init__(self,channel):
    super().__init__()
    self.block1=Block(channel)
    self.block2=Block(96)
    self.block3=Block(96)
    self.gru1=nn.LSTM(input_size=96,hidden_size=32,batch_first=True)
    self.gru2=nn.LSTM(input_size=32,hidden_size=32,batch_first=True)
    self.gru3=nn.LSTM(input_size=64,hidden_size=32,batch_first=True)
    self.gru4=nn.LSTM(input_size=96,hidden_size=32,batch_first=True)
    self.gru_linear=nn.Linear(62,1)
    self.flatten=nn.Flatten()
    self.fc1=nn.Linear(32,1)
    self.relu=nn.ReLU()

  def forward(self,x):
    x = x.squeeze()
    #print(x.shape)
    x=self.block1(x)
    #print(x.shape)
    x=self.block2(x)
    #print(x.shape)
    x=self.block3(x)
    #print(x.shape)
    x=x.permute(0,2,1)
    #print(x.shape)
    gru_out1,_=self.gru1(x)
    #print(gru_out1.shape)
    gru_out2,_=self.gru2(gru_out1)
    #print(gru_out2.shape)
    gru_out=torch.cat([gru_out1, gru_out2],dim=2)
    #print(gru_out.shape)
    gru_out3,_=self.gru3(gru_out)
    #print(gru_out3.shape)
    gru_out=torch.cat([gru_out1, gru_out2, gru_out3],dim=2)
    #print("aaa", gru_out.shape)
    linear_out=self.relu(self.gru_linear(gru_out.permute(0,2,1)))
    #print(linear_out.shape)
    gru_out4,_=self.gru4(linear_out.permute(0,2,1))
    #print(gru_out4.shape)
    x=self.flatten(gru_out4)
    #print(x.shape)
    x=self.fc1(x)
    return x


class LSTM(nn.Module):
    def __init__(self, input_size, num_channels, hidden_units):
        super().__init__()
        self.num_channels = num_channels
        self.hidden_units = hidden_units
        self.num_layers = 2
        self.input_size = input_size
        self.inputsize_channels = num_channels * input_size

        self.lstm = nn.LSTM(
            input_size=num_channels,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)
        self.enc = nn.Linear(in_features=self.inputsize_channels, out_features=19*num_channels)

    def forward(self, x):
        #print(x.shape)
        batch_size = x.shape[0]
        x = x.view(batch_size, x.shape[1]*x.shape[2])
        #print(x.shape)
        x = self.enc(x)
        #print(x.shape)
        x = x.view(batch_size, int(x.shape[-1]/self.num_channels), self.num_channels)
        #print(x.shape)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        lstm_out, (hn, _) = self.lstm(x, (h0, c0))
        #print(hn.shape)
        out = self.linear(hn[0]).flatten()
        #print(lstm_out.shape)
        #print(lstm_out.shape)
        #lstm_out = lstm_out.view(batch_size,int(lstm_out.shape[-1]/19), 19)
        #out = torch.sigmoid(out)
        return lstm_out

    
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        #self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        #if bias:
        #    self.bias = Parameter(torch.FloatTensor(out_features))
        #else:
        #    self.register_parameter('bias', None)
        #self.reset_parameters()
        
        self.LSTM = ChronoNet(19)#LSTM(input_size=500, num_channels=19, hidden_units=128)
        
    #def reset_parameters(self):
    #    stdv = 1. / math.sqrt(self.weight.size(1))
    #    self.weight.data.uniform_(-stdv, stdv)
    #    if self.bias is not None:
    #        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(adj, input)
        #print("INPUT", input.shape)
        output = self.LSTM(support)
        #print(adj.shape, support.shape)
        #support = torch.mm(input, self.weight)
        
        #if self.bias is not None:
        #    return output + self.bias
        #else:
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    
    
    
import torch.nn as nn
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nclass)
        #self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout
        self.flatten=nn.Flatten()
        self.fc1=nn.Linear(2432,1)
        self.relu=nn.ReLU()
        
    def forward(self, x, adj):
        x = self.gc1(x, adj)
        #x = F.dropout(x, self.dropout, training=self.training)
        #x = self.gc2(x, adj)
        #print(x.shape)
        #out = x.view(x.shape[0], x.shape[1]*x.shape[2])
        #out = self.fc1(out)
        #out = torch.sigmoid(out)
        return x

In [259]:
ch_net = ChronoNet(19)
ls = LSTM(500, 19, 32)
ch_net(features)
print("##########")
ls(features)

torch.Size([3, 19, 500])
torch.Size([3, 96, 250])
torch.Size([3, 96, 125])
torch.Size([3, 96, 62])
torch.Size([3, 62, 96])
torch.Size([3, 62, 32])
torch.Size([3, 62, 32])
torch.Size([3, 62, 64])
torch.Size([3, 62, 32])
aaa torch.Size([3, 62, 96])
torch.Size([3, 96, 1])
torch.Size([3, 1, 32])
torch.Size([3, 32])
##########


tensor([[[ 0.0679, -0.0494, -0.0534,  ...,  0.0110,  0.0654,  0.0448],
         [ 0.1034, -0.0800, -0.0946,  ...,  0.0159,  0.0880,  0.0647],
         [ 0.1202, -0.0915, -0.1146,  ...,  0.0091,  0.0978,  0.0736],
         ...,
         [ 0.1205, -0.1052, -0.1246,  ...,  0.0066,  0.0946,  0.0791],
         [ 0.1166, -0.1080, -0.1335,  ...,  0.0127,  0.0814,  0.0744],
         [ 0.1138, -0.1105, -0.1390,  ...,  0.0084,  0.0852,  0.0808]],

        [[ 0.0693, -0.0498, -0.0608,  ...,  0.0096,  0.0643,  0.0452],
         [ 0.1027, -0.0793, -0.0968,  ...,  0.0076,  0.0923,  0.0707],
         [ 0.1160, -0.0911, -0.1170,  ...,  0.0051,  0.1012,  0.0773],
         ...,
         [ 0.1190, -0.1097, -0.1316,  ...,  0.0085,  0.0847,  0.0769],
         [ 0.1159, -0.1112, -0.1352,  ...,  0.0168,  0.0696,  0.0688],
         [ 0.1118, -0.1129, -0.1383,  ...,  0.0069,  0.0786,  0.0773]],

        [[ 0.0709, -0.0497, -0.0579,  ...,  0.0101,  0.0649,  0.0422],
         [ 0.1102, -0.0792, -0.0971,  ...,  0

In [182]:
#adj.shape, features.shape
features = torch.rand(3, 19, 500)
adj = torch.rand(3, 19, 19)
aa = torch.matmul(adj, features)
#aa.shape

In [189]:
train_features.shape

(88500, 19, 500)

In [195]:
from scipy.stats import mstats
from tqdm import tqdm
"""
adj = []
for feat in train_features:
    for ch in feat:
        pcorr = mstats.pearsonr([1, 2, 3, 4, 5], [10, 9, 2.5, 6, 4])[0]
        adj.append(pcorr)
""" 

adj=[]
for f_t in tqdm(train_features):
    matrix = []
    for i in range(19):
        row = []
        for j in range(19):
            res = mstats.pearsonr(f_t[i, :], f_t[j, :])
            row.append(res[0])
        matrix.append(row)
        #matrix = np.array(matrix)
    adj.append(matrix)
adj=np.array(adj)

100%|█████████████████████████████████████| 88500/88500 [42:04<00:00, 35.06it/s]


In [239]:
val_adj = []
for f_t in tqdm(val_features):
    matrix = []
    for i in range(19):
        row = []
        for j in range(19):
            res = mstats.pearsonr(f_t[i, :], f_t[j, :])
            row.append(res[0])
        matrix.append(row)
        #matrix = np.array(matrix)
    val_adj.append(matrix)
val_adj = np.array(val_adj)

100%|█████████████████████████████████████| 29500/29500 [14:04<00:00, 34.94it/s]


In [276]:
test_adj = []
for f_t in tqdm(test_features):
    matrix = []
    for i in range(19):
        row = []
        for j in range(19):
            res = mstats.pearsonr(f_t[i, :], f_t[j, :])
            row.append(res[0])
        matrix.append(row)
        #matrix = np.array(matrix)
    test_adj.append(matrix)
test_adj = np.array(test_adj)

100%|█████████████████████████████████████| 32568/32568 [15:39<00:00, 34.67it/s]


In [242]:
adj.shape, train_features.shape, train_labels.shape, val_features.shape, val_adj.shape , val_labels.shape

(torch.Size([88500, 19, 19]),
 torch.Size([88500, 19, 500]),
 torch.Size([88500]),
 (29500, 19, 500),
 torch.Size([29500, 19, 19]),
 (29500,))

In [186]:
"""
import torch
import torch.nn.functional as F
import torch.optim as optim



optimizer = optim.Adam(model.parameters(), lr=0.001)

output = model(features, adj)
output.shape
"""

torch.Size([3, 19, 128])
torch.Size([3, 19, 19]) torch.Size([3, 19, 128])


torch.Size([3, 19, 128])

In [264]:
features = torch.rand(3, 19, 500)
adj = torch.rand(3, 19, 19)
aa = torch.matmul(adj, features)
aa.shape

torch.Size([3, 19, 500])

In [270]:
gcn = GCN(nfeat=features.shape[1], nhid=4, nclass=2, dropout=0.5)
out = gcn(features, adj)

In [243]:
#train_features = torch.Tensor(train_features).float().to(device)
#train_labels = torch.Tensor(train_labels).float().to(device)
adj = torch.Tensor(adj).float().to(device)
data = torch.utils.data.TensorDataset(train_features, adj, train_labels)
data_iter = torch.utils.data.DataLoader(data, 128, shuffle=True)

val_adj = torch.Tensor(val_adj).float().to(device)
val_labels = torch.Tensor(val_labels).float().to(device)
val_features = torch.Tensor(val_features).float().to(device)
data = torch.utils.data.TensorDataset(val_features, val_adj, val_labels)
val_data_iter = torch.utils.data.DataLoader(data, 128, shuffle=True)

In [277]:
test_adj = torch.Tensor(test_adj).float().to(device)
test_labels = torch.Tensor(test_labels).float().to(device)
test_features = torch.Tensor(test_features).float().to(device)
data = torch.utils.data.TensorDataset(test_features, test_adj, test_labels)
test_data_iter = torch.utils.data.DataLoader(data, 128, shuffle=True)

In [274]:
def cal_accuracy(model, data_iter):
    ytrue = []
    ypreds = []
    with torch.no_grad():
        for x, x_adj, y in data_iter:
            yhat = model(x, x_adj)
            yhat = [0 if i<0.5 else 1 for i in yhat]
            ytrue.extend(list(y.numpy()))
            ypreds.extend(yhat)

    return (accuracy_score(ytrue, ypreds), 
            confusion_matrix(ytrue, ypreds), 
            precision_score(ytrue, ypreds), 
            recall_score(ytrue, ypreds),
            f1_score(ytrue, ypreds))

def evaluate_model(model, loss_func, data_iter):
    model.eval()
    loss_sum, n = 0, 0
    with torch.no_grad():
        for x, x_adj, y in data_iter:
            y_pred = model(x, x_adj)
            y_pred = y_pred.squeeze()
            loss = loss_func(y_pred,y)
            loss_sum += loss.item()
            n += 1
        return loss_sum / n

In [275]:
print("Training Model....")
n_chans = 19
DEVICE = torch.device("cpu")
# Model and optimizer
gcn = GCN(nfeat=features.shape[1], nhid=4, nclass=2, dropout=0.5)
gcn.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(gcn.parameters(), lr=1e-3)
NUM_EPOCHS = 5
BATCH_SIZE = 128

print("started training")

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, adj_x ,y) in enumerate(tqdm(data_iter)):
        y_pred = gcn(x, adj_x)
        y_pred = y_pred.squeeze()
        #print(y_pred.shape, y.shape)
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(gcn, loss_func, val_data_iter)
    print("Train loss:", loss_sum / (t+1), ",Train Accuracy: ", 
        cal_accuracy(gcn, data_iter)[0], ",F1: ", 
        cal_accuracy(gcn, data_iter)[4], ",Precision: ", 
        cal_accuracy(gcn, data_iter)[2], ",Recall: ", 
        cal_accuracy(gcn, data_iter)[3])
    
    print("Val loss:", val_loss, ", Val Accuracy: ", 
        cal_accuracy(gcn, val_data_iter)[0], ",F1: ", 
        cal_accuracy(gcn, val_data_iter)[4], ",Precision: ", 
        cal_accuracy(gcn, val_data_iter)[2], ",Recall: ", 
        cal_accuracy(gcn, val_data_iter)[3])

Training Model....
started training
Epoch 1


100%|█████████████████████████████████████████| 692/692 [06:20<00:00,  1.82it/s]


Train loss: 0.4537571369740315 ,Train Accuracy:  0.8041920903954802 ,F1:  0.7702364063059359 ,Precision:  0.9318276603253024 ,Recall:  0.6564067796610169
Val loss: 0.5159348890636907 , Val Accuracy:  0.7381016949152542 ,F1:  0.6805061616078075 ,Precision:  0.8723494486853266 ,Recall:  0.5578305084745763
Epoch 2


100%|█████████████████████████████████████████| 692/692 [05:57<00:00,  1.94it/s]


Train loss: 0.3522290437336946 ,Train Accuracy:  0.8464632768361582 ,F1:  0.8281696552770681 ,Precision:  0.9401917997013897 ,Recall:  0.74
Val loss: 0.5635575216053884 , Val Accuracy:  0.7513220338983051 ,F1:  0.7108851580357847 ,Precision:  0.8489269578313253 ,Recall:  0.611457627118644
Epoch 3


...

Train loss: 0.30415976473878575 ,Train Accuracy:  0.8739096045197741 ,F1:  0.8781249658697481 ,Precision:  0.8497178246073852 ,Recall:  0.908497175141243
Val loss: 0.6039410845025793 , Val Accuracy:  0.7412203389830508 ,F1:  0.749573546778638 ,Precision:  0.7261344858268718 ,Recall:  0.7745762711864407
Epoch 4


100%|█████████████████████████████████████████| 692/692 [04:52<00:00,  2.36it/s]


Train loss: 0.2665473829976396 ,Train Accuracy:  0.8761920903954802 ,F1:  0.862699397265767 ,Precision:  0.968216465558462 ,Recall:  0.7779209039548023
Val loss: 0.619712154973637 , Val Accuracy:  0.7494237288135593 ,F1:  0.7085173501577287 ,Precision:  0.8467483506126295 ,Recall:  0.6090847457627119
Epoch 5


  2%|█                                         | 17/692 [00:07<04:57,  2.27it/s]


KeyboardInterrupt: 

In [279]:
from collections import Counter
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

ytrue = []
ypreds = []
with torch.no_grad():
    for x, test_adj, y in test_data_iter:
        yhat = gcn(x, test_adj)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)
        
y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)
    
print("Test Accuracy: ", accuracy_score(y_final, yhat_final))
print("Test Confusion: ",confusion_matrix(y_final, yhat_final))
print("Test F1: ", f1_score(y_final, yhat_final))
print("Test Recall: ", recall_score(y_final, yhat_final))
print("Test Precision: ", precision_score(y_final, yhat_final))

Test Accuracy:  0.8369565217391305
Test Confusion:  [[231   0]
 [ 45   0]]
Test F1:  0.0
Test Recall:  0.0
Test Precision:  0.0


/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
